In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# Reports System

## Default Usage

OpenMDAO has a reports system which will generate several common reports when you run your model. Currently, the only reports that are run are the N2 diagram and the scaling report.

The N2 diagram report contains an interactive, graphical view of the model that is viewable in a browser.

The scaling report contains tables of information for design variables, objectives, and constraints, as well as a viewer that shows magnitudes of subjacobians of the total jacobian.

The names of these reports in the reporting system, are "n2" and "scaling", respectively.

The n2 report is generated after `Problem.final_setup` runs.
The scaling report is run after `Driver._compute_totals`.

Note that the reports are only run the first time the method is run on a given instance. For example, the scaling report is only run after the first call to `Driver._compute_totals`.

These reports will, by default, be put into a folder named after the `Problem` name. For example, if your model
has a single Problem and its default name, `problem1`, was not changed by the user, the reports will be placed in a directory named `problem1`.

The user has control over this reports system by setting some environment variables which are described below.

## Controlling which reports get run

There are two ways to control which reports get run on which models.

### Controlling reporting using an environment variable
To turn off all the report generation that is part of the default report generation system of OpenMDAO,
the user should set an environment variable of `OPENMDAO_REPORTS` to any one of these four values: '0', 'false', 'off', or 'none'.

The user can also have more fine-grained control over what reports get run by setting this environment variable to a string that lists the reports to run. For example

1. OPENMDAO_REPORTS=n2

2. OPENMDAO_REPORTS=n2,scaling

3. OPENMDAO_REPORTS=none

If `OPENMDAO_REPORTS` is not set, all default reports are run. The user can also explicitly turn on all reports by setting `OPENMDAO_REPORTS` to any of these four values: '1', 'true', 'on', or 'all'.

### Controlling reporting using the Problem reports argument 
The user can also control what reports get run using the `reports` argument to `Problem`.

If `reports` is the default of `_UNDEFINED`, the `OPENMDAO_REPORTS` environment variable 
determines reporting.

If the value for `reports` is a Boolean, it will enable/disable all reports on that `Problem`. A value of `None` is equivalent to `False`.

Otherwise, `reports` may be a comma-delimted string of the names of the reports to run for that Problem.

## Setting the directory for the reports

If the user wishes to have the reports directory placed into a different directory other than the default current working
directory, the user can set the environment variable, `OPENMDAO_REPORTS_DIR` to the name of that directory. The
directory does not have to exist beforehand. It will be created by the reporting system. As an example, if `OPENMDAO_REPORTS_DIR` is set to "my_reports", then, if the `Problem` name is `problem1`, then the reports will be written to the "my_reports/problem1" directory.

## Support for subproblems

The reporting system supports subproblems. For example, if a model has two Problems and they have the default names of
`problem1` and `problem2`, the reporting system will put the reports into the folders, `problem1` and
`problem2`.

## Adding user defined reports

The user can register a user defined report with the reporting system using the `register_report` function.

```{eval-rst}
    .. autofunction:: openmdao.utils.reports_system.register_report
        :noindex:
```

Here is a script to show how this is done. Note that the function that is called to do the reporting must have
as its first argument, the class name of the OpenMDAO object owning the method where the report will be run. Currently, the only options are `Problem` and `Driver`. The second argument to the report function is a file path to where to write the report. The user defines the file name for their report in the argument to the decorator `report_function` that must be used. The decorator takes care of making sure the full file path to the report file is determined correctly.

After this script is run, there will be a file named `user_report.txt` in the directory `problem1`.

In [ ]:
import os
os.environ['OPENMDAO_REPORTS'] = '1'  # Need to run reports in this notebook.

In [ ]:
import pathlib
import openmdao.api as om
from openmdao.test_suite.components.paraboloid import Paraboloid
from openmdao.utils.reports_system import report_function

user_report_filename = 'user_report.txt'

@report_function()
def user_defined_report(prob, report_filepath):
    with open(report_filepath, "w") as f:
        f.write(f"Do some reporting on the Problem, {prob._name}\n")

# Run the report before the call to Problem.setup
om.register_report("user_report", user_defined_report, "This report is user defined",
                     'Problem', 'setup', 'pre', user_report_filename)

prob = om.Problem()
model = prob.model

model.add_subsystem('p1', om.IndepVarComp('x', 0.0), promotes=['x'])
model.add_subsystem('p2', om.IndepVarComp('y', 0.0), promotes=['y'])
model.add_subsystem('comp', Paraboloid(), promotes=['x', 'y', 'f_xy'])

model.add_design_var('x', lower=0.0, upper=1.0)
model.add_design_var('y', lower=0.0, upper=1.0)
model.add_objective('f_xy')

prob.driver = om.ScipyOptimizeDriver()

prob.setup()
prob.run_driver()
prob.cleanup()

In [ ]:
import pathlib
from openmdao.utils.reports_system import _reports_dir
problem_reports_dir = pathlib.Path(_reports_dir).joinpath(prob._name)

path = pathlib.Path(problem_reports_dir).joinpath(user_report_filename)
assert path.is_file(),"Could not find problem report"

## Listing registered reports

The user can use the `list_reports` function to see what reports are registered.

```{eval-rst}
    .. autofunction:: openmdao.utils.reports_system.list_reports
        :noindex:
```

In [ ]:
om.list_reports()

In [ ]:
os.environ['OPENMDAO_REPORTS'] = '0'